0_methods_B_tuning_2_constrain_param_sets.ipynb  
# Tuning: constrain parameter sets

**Notebook input:**
- parameters_2TU_sampled.csv (from subfolder 0_methods_B_tuning_files)
- parameters_3P5_sampled.csv (from subfolder 0_methods_B_tuning_files)

**Notebook output (written to savedir):**
- parameters_2TU_sampled_after_constraints.csv
- parameters_3P5_sampled_after_constraints.csv

**Notebook comments:**  
This notebook edits the sampled parameter sets. It does the following:
- I. change setIDS from 0, 1, ..., 3000 to 1000, 1001, ..., 4000 for practical reasons (runnames must have same length and trailing zeros like 0001 were deleted in slurm array jobs)
- II. [only for ensemble 2TU:] constrain the parameter sets to only keep combinations that make sense, based on the literature. Since applying these constrains in ensemble 2TU did not solve the issues with tuning all parameters simultaneously, this approach was discarded in later ensembles such as 3P5. Instead we used a 3-step tuning approach (1. ws, 2. kdes, 3. sigmas parameters).  
- Note: for ensemble 3P5, the resulting csv is also called _sampled_after_constraints.csv for consistency, even though no constraints were applied (only setIDs were changed).  
- Note: for ensemble KDE, no sampling was performed. Instead, the file parameters_KDE_sampled_after_constraints.csv was created manually directly.

**General information:**
- This notebook was published in the zenodo repository https://doi.org/10.5281/zenodo.10622403 along with the paper:

   - Jeemijn Scheen, Jörg Lippold, Frerk Pöppelmeier, Finn Süfke and Thomas F. Stocker. Promising regions for detecting the overturning circulation in Atlantic 231Pa/230Th: a model-data comparison. Paleoceanography and Paleoclimatology, 2025.
- Code author for this notebook: Jeemijn Scheen

# Set up notebook

**Easiest:** load conda environment for which this notebook worked.  
**Just use environment.yml (present in this folder) and follow** https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-from-an-environment-yml-file

In [1]:
from pathlib import Path         # Path objects to avoid inter-platform trouble in file paths
import platform

########## SET FILE PATHS ######################################
# KEEP THE Path() FUNCTION AND USE FORWARD SLASHES '/' ON EVERY OPERATING SYSTEM
obsdir = Path('./0_methods_B_tuning_files/')    # csv's are loaded from here
savedir = Path('./figures/')                    # csv's are written to here
#############################################

## CHECK FILEPATHS
# expand paths because np.loadtxt can't handle home directory ~
savedir = savedir.expanduser()
obsdir = obsdir.expanduser()
def check_dir(path):
    if not path.exists():
        raise Exception('File path ' + str(path) + ' does not exit. Correct or create first.')
check_dir(savedir)
check_dir(obsdir)

## IMPORT PACKAGES
# first time install missing packages via $conda install numpy OR $pip3 install numpy (be consistent)
import numpy as np
import xarray as xr                            # $conda install -c anaconda -n cartopy xarray; needs some time
import pandas as pd
import importlib as imp                        # to import user-defined functions; renaming new name to name of deprecated package 'imp'
import math                                    # math.e or math.exp()
import xesmf as xe                             # regridding; install via conda-forge channel e.g. !conda install -c conda-forge xesmf -y

# plot-related packages:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MultipleLocator
matplotlib.rcParams['savefig.bbox'] = 'tight'  # cuts off whitespace
import matplotlib.cm as cmp                    # colormaps
import cmcrameri.cm as cmcr                    # better colour maps (https://www.fabiocrameri.ch/colourmaps/); $conda install -c conda-forge cmcrameri
import cartopy.crs as ccrs
import seaborn as sns

## CHECK PYTHON VERSION
if 1/2 == 0:
    raise Exception("You are using python 2. Please use python 3 for a correct display of the figures.") 

## PLOT SETTINGS
# larger labels
params = {'legend.fontsize': 'x-large',
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

## LOAD USER-DEFINED FUNCTIONS
import functions as f                          # my own functions; call via f.function_name()

In [2]:
imp.reload(f)   # shows how to reload functions after editing functions.py w/o kernel restart

<module 'functions' from '/Users/jeemijnscheen/Documents/PHD/PhD_notebooks/functions.py'>

# Load parameter sets with pandas

In [3]:
## SETTINGS ##############
ensemble = '2TU'   # choose '2TU' or '3P5'
##########################

assert ensemble in ['2TU', '3P5'], "Choose '2TU' or '3P5' ensemble."
all_param_sets = pd.read_csv(obsdir / ("parameters_" + ensemble + "_sampled.csv"), 
                             index_col=None, encoding='utf-8-sig')  # encoding avoids \ufeff string in python3
all_param_sets # show

,# setID,BGC_PaThWs,BGC_PaDesConst,BGC_ThDesConst,BGC_sigmaPaPOC,BGC_sigmaPaCa,BGC_sigmaPaOp,BGC_sigmaPaDu,BGC_sigmaPaNeph,BGC_sigmaThPOC,BGC_sigmaThCa,BGC_sigmaThOp,BGC_sigmaThDu,BGC_sigmaThNeph
0,0,821.531393,1.048681,0.994627,0.015353,0.039203,0.023205,0.025231,0.018609,0.087232,6.804295,0.160851,0.014811,0.047647
1,1,768.728884,3.150576,4.082091,0.018461,0.153653,0.079377,0.006370,0.000517,0.079977,4.128426,0.149198,0.044603,0.037022
2,2,4413.960452,4.425443,4.295279,0.016316,0.184002,0.194621,0.017425,0.017216,0.121005,5.349578,0.095412,0.003024,0.024386
3,3,1620.657572,2.825875,3.412331,0.006355,0.197896,0.194282,0.022132,0.001262,0.113021,7.908430,0.123693,0.017201,0.013824
4,4,3233.954356,2.312236,1.839610,0.009044,0.009794,0.164971,0.019107,0.024428,0.121655,0.195806,0.173670,0.050808,0.031938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3220,3220,1795.517162,4.652725,1.408416,0.020889,0.160012,0.201081,0.026822,0.002617,0.042020,3.428338,0.134136,0.006598,0.041862
3221,3221,3173.757204,4.323337,1.597070,0.007048,0.161749,0.167592,0.013054,0.011322,0.139429,3.847583,0.029506,0.005838,0.050629
3222,3222,1707.468940,3.582293,2.393961,0.011686,0.048299,0.112772,0.023818,0.003603,0.102737,2.783510,0.199974,0.036087,0.029584
3223,3223,1718.582058,4.185762,3.316339,0.019844,0.110242,0.036037,0.005964,0.017704,0.046529,5.326045,0.114939,0.020750,0.016759


# Apply constraints
Only for ensemble 2TU.  

The constraints are:
- sigmaPaCa < sigmaThCa
- sigmaPaDu < sigmaThDu
- sigmaPaNeph < sigmaThNeph
- sigmaPaNeph < sigmaPaDu
- sigmaThNeph < sigmaThDu  

These constraints make the parameter sets we try out more in line with the literature, and also make the parameter space we explore smaller.

In [4]:
if ensemble == '2TU':
   constrained_param_sets = all_param_sets.where(
      all_param_sets.BGC_sigmaPaCa < all_param_sets.BGC_sigmaThCa).where(
      all_param_sets.BGC_sigmaPaDu < all_param_sets.BGC_sigmaThDu).where(
      all_param_sets.BGC_sigmaPaNeph < all_param_sets.BGC_sigmaThNeph).where(
      all_param_sets.BGC_sigmaPaNeph < all_param_sets.BGC_sigmaPaDu).where(
      all_param_sets.BGC_sigmaThNeph < all_param_sets.BGC_sigmaThDu).dropna()
elif ensemble == '3P5':
   # not apply any constraints
   constrained_param_sets = all_param_sets

constrained_param_sets # show

,# setID,BGC_PaThWs,BGC_PaDesConst,BGC_ThDesConst,BGC_sigmaPaPOC,BGC_sigmaPaCa,BGC_sigmaPaOp,BGC_sigmaPaDu,BGC_sigmaPaNeph,BGC_sigmaThPOC,BGC_sigmaThCa,BGC_sigmaThOp,BGC_sigmaThDu,BGC_sigmaThNeph
1,1.0,768.728884,3.150576,4.082091,0.018461,0.153653,0.079377,0.006370,0.000517,0.079977,4.128426,0.149198,0.044603,0.037022
6,6.0,3649.788583,4.620634,3.508209,0.004668,0.038705,0.081266,0.022354,0.006533,0.044272,0.357777,0.178698,0.048421,0.011153
7,7.0,664.343732,3.068190,4.196783,0.018889,0.030927,0.097789,0.001390,0.000948,0.031445,2.733750,0.172761,0.022869,0.004043
16,16.0,4043.612841,3.870075,2.976250,0.015599,0.112339,0.192899,0.011735,0.009383,0.111021,4.933552,0.024748,0.033035,0.023275
26,26.0,2844.585771,4.688713,3.518719,0.008270,0.194907,0.199750,0.021821,0.002775,0.103188,5.338171,0.176707,0.051654,0.050282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,3195.0,3519.936961,2.324750,4.506496,0.011727,0.093204,0.071221,0.013308,0.009869,0.127985,2.655257,0.163128,0.042854,0.035743
3205,3205.0,1082.988226,2.795878,2.180482,0.016019,0.001830,0.037924,0.024263,0.012562,0.125422,4.710938,0.158923,0.039041,0.019041
3206,3206.0,2317.522048,1.887593,2.276892,0.017501,0.043019,0.153382,0.014338,0.000152,0.032458,1.566325,0.195424,0.045282,0.025602
3218,3218.0,4354.432140,4.011962,3.225122,0.020015,0.177111,0.112027,0.020800,0.017095,0.070103,6.474240,0.045432,0.047012,0.036577


Tuning 1: So we indeed read in 7200 parameter sets, and after the constraints 2951 (ca. 3000) are left.  
Note there are now gaps in between the setIDs as expected. We need to rename them to 0,2,3,...,n-1 in order for our run procedure to understand how to read the csv file.  

In [5]:
constrained_param_sets.columns

Index(['# setID', 'BGC_PaThWs', 'BGC_PaDesConst', 'BGC_ThDesConst',
       'BGC_sigmaPaPOC', 'BGC_sigmaPaCa', 'BGC_sigmaPaOp', 'BGC_sigmaPaDu',
       'BGC_sigmaPaNeph', 'BGC_sigmaThPOC', 'BGC_sigmaThCa', 'BGC_sigmaThOp',
       'BGC_sigmaThDu', 'BGC_sigmaThNeph'],
      dtype='object')

# Update setIDs

In [6]:
n = len(constrained_param_sets)
# at the same time add 1000 to the setIDs because slurm array jobs delete trailing zeros (0001 becomes 1) thereby changing names of output files etc.
constrained_param_sets['# setID'] = range(1000,n+1000)
constrained_param_sets # show

,# setID,BGC_PaThWs,BGC_PaDesConst,BGC_ThDesConst,BGC_sigmaPaPOC,BGC_sigmaPaCa,BGC_sigmaPaOp,BGC_sigmaPaDu,BGC_sigmaPaNeph,BGC_sigmaThPOC,BGC_sigmaThCa,BGC_sigmaThOp,BGC_sigmaThDu,BGC_sigmaThNeph
1,1000,768.728884,3.150576,4.082091,0.018461,0.153653,0.079377,0.006370,0.000517,0.079977,4.128426,0.149198,0.044603,0.037022
6,1001,3649.788583,4.620634,3.508209,0.004668,0.038705,0.081266,0.022354,0.006533,0.044272,0.357777,0.178698,0.048421,0.011153
7,1002,664.343732,3.068190,4.196783,0.018889,0.030927,0.097789,0.001390,0.000948,0.031445,2.733750,0.172761,0.022869,0.004043
16,1003,4043.612841,3.870075,2.976250,0.015599,0.112339,0.192899,0.011735,0.009383,0.111021,4.933552,0.024748,0.033035,0.023275
26,1004,2844.585771,4.688713,3.518719,0.008270,0.194907,0.199750,0.021821,0.002775,0.103188,5.338171,0.176707,0.051654,0.050282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,1506,3519.936961,2.324750,4.506496,0.011727,0.093204,0.071221,0.013308,0.009869,0.127985,2.655257,0.163128,0.042854,0.035743
3205,1507,1082.988226,2.795878,2.180482,0.016019,0.001830,0.037924,0.024263,0.012562,0.125422,4.710938,0.158923,0.039041,0.019041
3206,1508,2317.522048,1.887593,2.276892,0.017501,0.043019,0.153382,0.014338,0.000152,0.032458,1.566325,0.195424,0.045282,0.025602
3218,1509,4354.432140,4.011962,3.225122,0.020015,0.177111,0.112027,0.020800,0.017095,0.070103,6.474240,0.045432,0.047012,0.036577


# Export as .csv

In [7]:
# export without the index column
constrained_param_sets.to_csv(savedir / ("parameters_" + ensemble + "_sampled_after_constraints.csv"), 
                              sep=',', header=True, index=False)